In [1]:
import duckdb

In [3]:
cn = duckdb.connect()
cn.execute("""
    create or replace view v_orders as select * from read_parquet('./data/orders.parquet');        
    create or replace view v_customers as select * from read_parquet('./data/customer_insights.parquet');
""")

In [9]:
# dup check
cn.sql("""
    select order_id
    from v_orders
    group by 1
    having count(*)>=2
""").show()

cn.sql("""
    select order_id, cust_id
    from v_customers
    group by all
    having count(*)>=2
""").show()

┌──────────┐
│ order_id │
│  int64   │
├──────────┤
│  0 rows  │
└──────────┘

┌──────────┬─────────┐
│ order_id │ cust_id │
│  int64   │  int32  │
├──────────┴─────────┤
│       0 rows       │
└────────────────────┘



In [11]:
cn.sql("""
    select o.order_id, o.cust_id, o.channel, c.survey_date, c.overall_satisfaction
    from v_orders as o
    left join v_customers as c
        on o.order_id = c.order_id and o.cust_id = c.cust_id   
    limit 10
""").show()

┌──────────┬─────────┬──────────┬─────────────┬──────────────────────┐
│ order_id │ cust_id │ channel  │ survey_date │ overall_satisfaction │
│  int64   │  int32  │ varchar  │    date     │        int32         │
├──────────┼─────────┼──────────┼─────────────┼──────────────────────┤
│        1 │    1505 │ Phone    │ 2025-03-30  │                    7 │
│        2 │    1146 │ In-Store │ 2025-08-15  │                    8 │
│        3 │    2549 │ Phone    │ 2025-11-09  │                    9 │
│        4 │     935 │ In-Store │ 2025-03-05  │                    7 │
│        5 │     200 │ Online   │ 2025-09-29  │                   10 │
│        6 │    2589 │ Phone    │ 2025-11-10  │                    8 │
│        7 │    2518 │ In-Store │ 2025-05-19  │                    9 │
│        8 │    1301 │ In-Store │ 2025-08-11  │                    8 │
│        9 │    2106 │ Online   │ 2025-11-17  │                    7 │
│       10 │    2195 │ In-Store │ 2025-04-16  │                    5 │
├─────